In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset

dataset = load_dataset("imdb")

print(dataset)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

plain_text/unsupervised-00000-of-00001.p(…):   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


In [ ]:
print(dataset["train"][0])

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [ ]:
!pip install transformers

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

text = "This movie is absolute trash"

tokens = tokenizer(text)

print(tokens)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

{'input_ids': [101, 2023, 3185, 2003, 7619, 11669, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}


In [ ]:
import torch
import torch.nn as nn

vocab_size = tokenizer.vocab_size
embedding_dim = 128

embedding = nn.Embedding(vocab_size, embedding_dim)

input_ids = torch.tensor(tokens["input_ids"])

embedded = embedding(input_ids)

print(embedded.shape)

torch.Size([7, 128])


In [ ]:
d_model = 128
W_q = nn.Linear(d_model, d_model)
W_k = nn.Linear(d_model, d_model)
W_v = nn.Linear(d_model, d_model)

Q = W_q(embedded)
K = W_k(embedded)
V = W_v(embedded)

print(Q.shape)

torch.Size([7, 128])


In [ ]:
import math

scores = torch.matmul(Q, K.transpose(0,1)) / math.sqrt(d_model)

print(scores.shape)

torch.Size([7, 7])


In [ ]:
attention_weights = torch.softmax(scores, dim=-1)

print(attention_weights.shape)

torch.Size([7, 7])


In [ ]:
output = torch.matmul(attention_weights, V)

print(output.shape)

torch.Size([7, 128])


In [ ]:
cls_vector = output[0]   # 取第一个token
print(cls_vector.shape)

torch.Size([128])


In [ ]:
classifier = nn.Linear(128, 2)

logits = classifier(cls_vector)

print(logits)

tensor([-0.1846, -0.1813], grad_fn=<ViewBackward0>)


In [ ]:
prediction = torch.argmax(logits)

print(prediction)

tensor(1)


In [ ]:
import torch
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 只取一部分数据加快训练
train_data = dataset["train"].select(range(2000))
test_data = dataset["test"].select(range(1000))

In [ ]:
def tokenize(example):
    return tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=128
    )

train_data = train_data.map(tokenize, batched=True)
test_data = test_data.map(tokenize, batched=True)

train_data.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
test_data.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32)

In [ ]:
import torch.nn as nn

class SimpleTransformer(nn.Module):
    def __init__(self, vocab_size, d_model=128, nhead=4):
        super().__init__()

        self.embedding = nn.Embedding(vocab_size, d_model)

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            batch_first=True
        )

        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=2)

        self.fc = nn.Linear(d_model, 2)

    def forward(self, input_ids):
        x = self.embedding(input_ids)
        x = self.transformer(x)

        cls_token = x[:, 0, :]   # 取CLS
        out = self.fc(cls_token)

        return out

In [ ]:
model = SimpleTransformer(tokenizer.vocab_size).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
for epoch in range(2):
    model.train()
    total_loss = 0

    for batch in train_loader:
        input_ids = batch["input_ids"].to(device)
        labels = batch["label"].to(device)

        optimizer.zero_grad()

        outputs = model(input_ids)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader)}")

Epoch 1, Loss: 0.009082396956006367
Epoch 2, Loss: 8.449801137589568e-05


In [ ]:
def predict(text):
    model.eval()

    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=128)
    input_ids = inputs["input_ids"].to(device)

    with torch.no_grad():
        outputs = model(input_ids)
        pred = torch.argmax(outputs, dim=1).item()

    return "Positive" if pred == 1 else "Negative"

In [ ]:
print(predict("This movie is absolute trash"))
print(predict("Masterpiece"))

Negative
Negative


In [ ]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch["input_ids"].to(device)
        labels = batch["label"].to(device)

        outputs = model(input_ids)
        preds = torch.argmax(outputs, dim=1)

        correct += (preds == labels).sum().item()
        total += labels.size(0)

print("Test Accuracy:", correct / total)

Test Accuracy: 1.0


In [ ]:
print(id(train_data))
print(id(test_data))

136327856047024
136327838217184


In [ ]:
print(train_data[0]["input_ids"][:10])
print(test_data[0]["input_ids"][:10])

tensor([  101,  1045, 12524,  1045,  2572,  8025,  1011,  3756,  2013,  2026])
tensor([  101,  1045,  2293, 16596,  1011, 10882,  1998,  2572,  5627,  2000])


In [ ]:
# 查看测试集标签分布
labels = [example["label"] for example in test_data]
print("Positive count:", sum(labels))
print("Total:", len(labels))

Positive count: tensor(0)
Total: 1000


In [ ]:
import random

indices = random.sample(range(len(dataset["test"])), 1000)
test_data = dataset["test"].select(indices)

test_data = test_data.map(tokenize, batched=True)
test_data.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

test_loader = DataLoader(test_data, batch_size=32)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch["input_ids"].to(device)
        labels = batch["label"].to(device)

        outputs = model(input_ids)
        preds = torch.argmax(outputs, dim=1)

        correct += (preds == labels).sum().item()
        total += labels.size(0)

print("Test Accuracy:", correct / total)

Test Accuracy: 0.499


In [ ]:
from transformers import pipeline

classifier = pipeline(
    "sentiment-analysis",
    model="textattack/bert-base-uncased-imdb"
)

Loading weights:   0%|          | 0/201 [00:00<?, ?it/s]

In [ ]:
print(classifier("This movie is absolute trash"))
print(classifier("Masterpiece"))

[{'label': 'LABEL_0', 'score': 0.991809070110321}]
[{'label': 'LABEL_1', 'score': 0.7475458383560181}]


In [ ]:
def predict(text):
    result = classifier(text)[0]
    label = result["label"]

    if label == "LABEL_0":
        sentiment = "Negative"
    else:
        sentiment = "Positive"

    print(f"Text: {text}")
    print(f"Sentiment: {sentiment}")
    print(f"Confidence: {result['score']:.4f}")
    print("-" * 40)

predict("This movie is absolute trash")
predict("Masterpiece")

Text: This movie is absolute trash
Sentiment: Negative
Confidence: 0.9918
----------------------------------------
Text: Masterpiece
Sentiment: Positive
Confidence: 0.7475
----------------------------------------
